In [1]:
import pandas as pd
import numpy as np
from numpy import log2
import pprint

# Dataset

In [2]:
outlook = 'overcast,overcast,overcast,overcast,rainy,rainy,rainy,rainy,rainy,sunny,sunny,sunny,sunny,sunny'.split(',')
temp = 'hot,cool,mild,hot,mild,cool,cool,mild,mild,hot,hot,mild,cool,mild'.split(',')
humidity = 'high,normal,high,normal,high,normal,normal,normal,high,high,high,high,normal,normal'.split(',')
windy = 'FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE'.split(',')
play = 'yes,yes,yes,yes,yes,yes,no,yes,no,no,no,no,yes,yes'.split(',')

# Create DataFrame

In [3]:
data={'outlook':outlook,'temperature':temp,'humidity':humidity,'windy':windy,'play':play}
dataset=pd.DataFrame(data,columns=['outlook','temperature','humidity','windy','play'])
dataset

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,FALSE,yes
1,overcast,cool,normal,TRUE,yes
2,overcast,mild,high,TRUE,yes
3,overcast,hot,normal,FALSE,yes
4,rainy,mild,high,FALSE,yes
5,rainy,cool,normal,FALSE,yes
6,rainy,cool,normal,TRUE,no
7,rainy,mild,normal,FALSE,yes
8,rainy,mild,high,TRUE,no
9,sunny,hot,high,FALSE,no


# Calculate  entropy H(s)
<font size ="3"> We consider the problem has C class. Assuming that when we work with non-leaf nodes with S data point set, |S|=N. In N data point, it has $N_c$ point belong to class c. </font>
    
 <br>
 <font size ="3"> 
    ==> Probability 1 point belongs to the class c is: \begin{equation*}\frac{ N_c}{N} \end{equation*} (Maximum likelihood estimation).So entropy H(s) is:  </font>
<img src="1.png" width="400" height="400">

In [4]:
def entropy(ds):
    entropy=0.0
    name_column=ds.keys()[-1]# return play
    labels=ds[name_column].unique() # return ['yes','no']
    
    for label in labels:
        fraction=ds[name_column].value_counts()[label]/len(ds[name_column])
        entropy-=fraction*log2(fraction)
    return entropy

# Calculate H(x,S)
<font size="3"> In each attribute x, data points in S was divided into k child-node $S_1$,$S_2$,...,$S_k$, each node has $m_1$,$m_2$,...,$m_k$ data points </font>
<img src="2.png" width="400" height="400">

<font size ="4" font color='red'> eps is smallest representable number. At times, we get log2(0) or 0 in the denominator, to avoid that we are going to use it </font>

In [5]:
eps=np.finfo(float).eps

def entropy_attribute(ds,attribute):
    entropy=0
    labels=ds[ds.keys()[-1]].unique()#return 'yes' and 'no'
    features=ds[attribute].unique()
    
    for feature in features:
        entr=0
        for label in labels:
            numerator=len(ds[attribute][ds[attribute]==feature][ds.play==label])
            denominator=len(ds[attribute][ds[attribute]==feature])
            fraction=numerator/(denominator)
            entr=-fraction*log2(fraction+eps)
        
        entropy+=(denominator/len(ds))*entr
    
    return entropy
        
    

In [6]:
def get_subtable(ds,node,attribute):
    return ds[ds[node]==attribute].reset_index(drop=True)

# Find attribute for node (non-leaf)
<font size ="3">In the below function, we find attribute x which has max Gain Information G(x,S)</font>
<img src="1.png" width="400" height="400">
<img src="2.png" width="400" height="400">
<br>
$$G(x,S)=H(s)-H(x,S)$$<br>

$$x= \underset{x}{\arg\max} G(x,S) <==> \underset{x}{\arg\min} H(x,S)$$

In [7]:
def find_non_leaf(ds):
    information_gain=[]
    name_columns=ds.keys()[:-1]
    
    for name in name_columns:
        information_gain.append(entropy(ds)-entropy_attribute(ds,name))
        
    return name_columns[np.argmax(information_gain)]

# Build decision tree

In [8]:
def build_tree(ds,tree=None):
    node=find_non_leaf(ds)
    if tree is None:
        tree={}
        tree[node]={}
    
    attribute_list=np.unique(ds[node])
    for attribute in attribute_list:
        subtable=get_subtable(ds,node,attribute)
        '''
        labels_list: return list which has all result in dataset['play'] correspond with attribute 
        counts: return list has number 'yes', number 'no'
        '''
        label_list,counts=np.unique(subtable['play'],return_counts=True)
        
        if len(counts)==1:# is purity subset
            tree[node][attribute]=label_list[0]
        
        else:
            tree[node][attribute]=build_tree(subtable)
            
    return tree

decision_tree=build_tree(dataset)
pprint.pprint(decision_tree)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {'FALSE': 'yes', 'TRUE': 'no'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}
